# Assignment - Antti Viljakainen

## Tehtävä 1

Optimoitava funktio on muotoa $\max a1 + a2$, jossa $a1$ on alla olevan neliön ala ja $a2$ puolikkaan ympyrän ala. Jos $x_1$ on neliön kanta ja $x_2$ sen sivu, sen ala olisi luonnollisesti $x_1x_2$, mutta on helpompaa jakaa $x_1$ kahteen puolikkaaseen, jolloin ympyrän ala on helpompi laskea, eli neliön ala on $2x_1x_2$. Puoliympyrän ala on $\frac12\pi x_1^2$. Rajoite-epäyhtälöllä otetaan huomioon käytettävissä olevan materiaalin määrä neliön kolmen sivun ja puoliympyrän kaaren mukaan $2x_1+2x_2+\pi x_1 \leq 12$

Optimointitehtävä kokonaisuudessaan on siis
$$
\max \frac12\pi x_1^2 + 2x_1x_2\\
\text{s.t. }2x_1+2x_2+\pi x_1 \leq 12
$$
Ja koska yleensä halutaan minimoida, saadaan
$$
\min -\frac12\pi x_1^2 - 2x_1x_2\\
\text{s.t. }-2x_1-2x_2-\pi x_1 \geq -12
$$

In [1]:
from math import pi
def f_constrained(x):
    return -0.5 * pi * x[0] ** 2 - 2 * x[0] * x[1], [12 -2 * x[0] - 2 * x[1] - pi * x[0]], []

def alpha(x,f):
    (_,ieq,eq) = f(x)
    return sum([min([0,ieq_j])**2 for ieq_j in ieq])+sum([eq_k**2 for eq_k in eq])

def f_penalized(f, x, r):
    return f(x)[0] + r * alpha(x, f)

In [2]:
from scipy.optimize import minimize
res = minimize(lambda x : f_penalized(f_constrained, x, 100000),
               [0, 0], method = 'Nelder-Mead', options={'disp': True})
print(res.x)

Optimization terminated successfully.
         Current function value: -10.081792
         Iterations: 138
         Function evaluations: 264
[ 1.68028702  1.68032849]


In [3]:
f_constrained(res.x)

(-10.081798850008376, [-8.3837994102609059e-06], [])

In [4]:
from pyomo.environ import *
from math import pi

model = ConcreteModel()
model.x = Var([1, 2], domain = NonNegativeReals)
model.OBJ = Objective(expr = -0.5 * pi * model.x[1] ** 2 - 2 * model.x[1] * model.x[2])
model.Constraint1 = Constraint(expr = -2 * model.x[1] - 2 * model.x[2] - pi * model.x[1] >= -12)

In [5]:
opt = SolverFactory("ipopt", solver_io = "nl")
opt.solve(model, tee = False)

model.x.display()

x : Size=2, Index=x_index, Domain=NonNegativeReals
    Key : Lower : Value              : Upper : Fixed : Stale
      1 :     0 :  1.680297476800643 :  None : False : False
      2 :     0 : 1.6802974779724507 :  None : False : False


Optimointiongelman ratkaisu on, että ikkunan leveys on 1.68 * 2 = 3.36m ja korkeus (kaaren yläreunaan) = 1.68 (neliön korkeus) + 1.68 (kaaren säde) = 3.36 metriä. Näin ollen ikkunan pinta-ala on noin 10m2. Samaan ratkaisuun päästään sekä ipopt:a käyttäen, että simppelillä penalty-funktioidealla.

## Tehtävä 2

Koska on kyse rajoitteettomasta optimointitehtävästä, voidaan rajata käytettävistä algoritmeista pois sellaiset, jotka erikoistuvat rajoittellisten optimointitehtävien ratkaisemiseen. Nelder-Mead on tässä tapauksessa mielenkiintoinen valinta verrokiksi, koska se perustuu pelkän objektifunktion arvon tarkastelemiseen. Muiksi verrokeiksi valitsin Powell:n, BFGS:n ja Newton-CG:n.

In [6]:
def rosenbrock(x):
    rsum = 0
    for i in range(0, len(x) - 1):
        rsum += 100 * (x[i + 1] - x[i] ** 2) ** 2 + (1 - x[i]) ** 2
    return rsum

def rosenbrock_gradient(x):
    return ad.gh(rosenbrock)[0](x);

In [7]:
print("Test rosenbrock: ", rosenbrock([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

Test rosenbrock:  0


In [8]:
from scipy.optimize import minimize
import ad
import time

initial_x = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
display = False

start = time.clock()
res_nm = minimize(rosenbrock, initial_x, method = 'Nelder-Mead', options = { 'disp' : display })
res_nm.runtime = time.clock() - start

start = time.clock()
res_pow = minimize(rosenbrock, initial_x, method = 'Powell', options = { 'disp' : display })
res_pow.runtime = time.clock() - start

start = time.clock()
res_ncg = minimize(rosenbrock, initial_x, method = 'Newton-CG', jac = lambda x: rosenbrock_gradient(x), options = { 'disp' : display })
res_ncg.runtime = time.clock() - start

start = time.clock()
res_bfgs = minimize(rosenbrock, initial_x, method = 'BFGS', jac = lambda x: rosenbrock_gradient(x), options = { 'disp' : display })
res_bfgs.runtime = time.clock() - start

res_nm["method"] = "Nelder-Mead"
res_pow["method"] = "Powell"
res_ncg["method"] = "Newton-CG"
res_bfgs["method"] = "BFGS"

res = [res_nm, res_pow, res_ncg, res_bfgs]

In [9]:
fmt = "{:>15}"
print(fmt.format("Metodi"), "|", fmt.format("# f(x)"), "|", fmt.format("# iter"), "|", fmt.format("Aika"))
print("-" * 75)
for r in res:
    print(fmt.format(r.method), "|", fmt.format(r.nfev), "|", fmt.format(r.nit), "|", fmt.format(r.runtime))

         Metodi |          # f(x) |          # iter |            Aika
---------------------------------------------------------------------------
    Nelder-Mead |            2000 |            1458 | 0.09391232000000001
         Powell |            6422 |              55 | 0.2035387733333333
      Newton-CG |              62 |              53 | 8.051857493333333
           BFGS |              81 |              64 | 0.9962244266666662


Päätelmiä: BFGS ei suotta taida olla oletusmetodi monissa optimointikirjastoissa, sillä se tarjoaa aika hyvän kompromissin kaikilla osa-alueilla tämän testin perusteella. Newton-CG suoriutuu vähemmillä iteraatioiden ja objektifunktiokutsujen määrillä, mutta sen suoritusaika on huomattavasti heikompi kuin BFGS:n. Powell:lla sentään löydetään maaliin, mutta funktiokutsujen määrästä voi päätellä, että metodi on vaikeuksissa, jos objektifunktio on vähänkään raskaampi laskettava. Nelder-Mead ei löydä edes optimia annetuissa rajoissa; suunta on oikea, mutta konvergointi niin hidasta, ettei sovellu tämän ongelman ratkaisemiseen.

## Tehtävä 3

Black box -tehtävässä sain vastaukseksi [1, 0, 0, 0] aloituspisteestä [0, 0, 0, 0] ja muutamalla muulla aloituspisteellä päästiin samaan tulokseen. f([1, 0, 0, 0]) = 3. Käytin lopulta penaltyfunktiota tähän, koska sillä saadaan tällaisessa black-box -tilanteessa yksinkertainen algoritmi aikaiseksi (ainakin annetuilla tiedoilla). Sakotin siis suoraan siitä tiedosta, jos ratkaisu ei ollut sallittu.

Scipy.optimizen dokumentaation mukaan yritin ensin saada SLSQP:n toimimaan, koska se on niitä harvoja, jotka (ainakin dokumentaation mukaan) tukevat constraints -parametria. Tulokset olivat kuitenkin laihoja kun metodi tökkäsi aina kääntymättömään matriisiin. Tein jopa niin, että etsin optimoinnilla sallitun vastauksen aloituspisteeksi (penalty-idealla), mutta siitäkään huolimatta ei onnistanut (vaikka tämä metodi itseasiassa antoi lähtöpisteeksi optimin). Jätän myös tähän yritykseen liittyviä koodeja tähän mukaan, mutta ne eivät siis johda oikeaan tulokseen.

In [10]:
import json
import urllib
from urllib.request import urlopen

def loaddata(x):
    params = [format(xi, '.10f') for xi in x]
    url = "http://mhartikainen.pythonanywhere.com/evaluate/{}/{}/{}/{}".format(*params)
    response = urlopen(url)
    datastr = response.read().decode('utf-8')
    return json.loads(datastr)

In [11]:
import numpy as np

def task3_fval(data):
    return data["function value"]
def task3_eqc(data):
    return data["equality constraint values"]
def task3_ineqc(data):
    return data["inequality constraint values"]

def task3_eqc_penalty(data):
    eqc = task3_eqc(data)
    return np.dot(eqc, eqc)
def task3_ineqc_penalty(data):
    return sum(np.power(np.minimum(task3_ineqc(data), 0), 2))

def task3_penalized(x):
    data = loaddata(x)
    
    fval = task3_fval(data)
    eqc_penalty = task3_eqc_penalty(data)
    ineqc_penalty = task3_ineqc_penalty(data)
    r = 10000
    
    return fval + r * (eqc_penalty + ineqc_penalty)

initial_x = [0, 0, 0, 0]

result = minimize(task3_penalized, 
         initial_x, 
         method = 'BFGS',
         options = {'disp' : True})

data = loaddata(result.x)
print("Optimal x: ", result.x)
print("f(x) at optimal: ", task3_fval(data))
print("Equality constraint values at optimal: ", task3_eqc(data))
print("Inequality constraint values at optimal: ", task3_ineqc(data))

Optimization terminated successfully.
         Current function value: 2.999800
         Iterations: 4
         Function evaluations: 42
         Gradient evaluations: 7
Optimal x:  [  9.99900003e-01  -3.36730497e-09  -3.36730500e-09  -3.36730500e-09]
f(x) at optimal:  2.9995999995995004
Equality constraint values at optimal:  [0.00010000769999984893, -9.999750000000418e-05]
Inequality constraint values at optimal:  [1.0000000068000001]


In [12]:
# X pyöristettynä

test_x = [1, 0, 0, 0]
data = loaddata(test_x)
print("Optimal x: ", test_x)
print("f(x) at optimal: ", task3_fval(data))
print("Equality constraint values at optimal: ", task3_eqc(data))
print("Inequality constraint values at optimal: ", task3_ineqc(data))

Optimal x:  [1, 0, 0, 0]
f(x) at optimal:  3.0
Equality constraint values at optimal:  [0.0, 0.0]
Inequality constraint values at optimal:  [1.0]


##### Alla olevat eivät liity ratkaisuun ks. kommentit

In [13]:
# Lähtöpiste SLSQP:lle - käytetään penalty-ideaa hyväksi etsiessä sallittu lähtöpiste

def feasiblex_objf(x):
    data = loaddata(x)
    
    eqc_penalty = task3_eqc_penalty(data)
    ineqc_penalty = task3_ineqc_penalty(data)
    
    return eqc_penalty + ineqc_penalty

initial_x = [0, 0, 0, 0]
res = minimize(feasiblex_objf, initial_x, options = {'disp' : False})

data = loaddata(res.x)
print("Feasible x: ", res.x)
print("Infeasibility penalty: ", feasiblex_objf(res.x))
print("Equality constraint values at optimal: ", task3_eqc(data))
print("Inequality constraint values at optimal: ", task3_ineqc(data))

Feasible x:  [  9.99999911e-01  -4.12614661e-08  -4.12614661e-08  -4.02956424e-08]
Infeasibility penalty:  5.31240099607e-14
Equality constraint values at optimal:  [2.123999999215087e-07, -8.949999996676894e-08]
Inequality constraint values at optimal:  [1.0000000816]


In [14]:
def readconstraintdescription(data):
    constraints = ()
    
    for index, value in enumerate(data["inequality constraint values"]):
        constraints = constraints + ({'type' : 'ineq',\
                         'fun' : lambda x : loaddata(x)["inequality constraint values"],\
                         'jac' : lambda x : loaddata(x)["inequality constraint gradient"]},)
    
    for index_eq, value in enumerate(data["equality constraint values"]):
        constraints = constraints + ({'type' : 'eq',\
                         'fun' : lambda x : loaddata(x)["equality constraint values"][index_eq],\
                         'jac' : lambda x : loaddata(x)["equality constraint gradients"][index_eq]},)
    
    return constraints

initial_x = [1, 0, 0, 0]
data = loaddata(initial_x)
constraints = readconstraintdescription(data)

minimize(lambda x : loaddata(x)["function value"], 
         initial_x, 
         method = 'SLSQP',
         jac = lambda x : loaddata(x)["function gradient"],
         constraints = constraints, 
         options = {'disp' : True})

Singular matrix C in LSQ subproblem    (Exit mode 6)
            Current function value: 3.0
            Iterations: 1
            Function evaluations: 1
            Gradient evaluations: 1


     nit: 1
  status: 6
       x: array([ 1.,  0.,  0.,  0.])
 message: 'Singular matrix C in LSQ subproblem'
    nfev: 1
    njev: 1
     jac: array([ 4.,  2.,  2.,  2.,  0.])
     fun: 3.0
 success: False

## Tehtävä 4

Kyseessä on vastaava tehtävä kuin harjoituksessa 6. Molemmat optimoitavat funktiot saadaan minimoitua nollaan käyttämällä ensimmäiselle funktiolle vektoria $(1, 0)$ ja toiselle $(0, 1)$.

Näiden avulla voidaan selvittää myös $z^{nadir}$, joka on käytännössä 
$$
(\min f_1 \text{ s.t. } f_2(x) \leq z_2^{ideal}, \min f_2 \text{ s.t. } f_1(x) \leq z_1^{ideal})\\
(\min f_1 \text{ s.t. } f_2(x) = 0, \min f_2 \text{ s.t. } f_1(x) = 0)\\
(f_1((0, 1)), f_2((1, 0))) = (\sqrt{2}, \sqrt{2})
$$

Näin ollen 
$$z^{ideal}=(0, 0), z^{nadir}=(\sqrt{2}, \sqrt{2})$$

In [15]:
import matplotlib.pyplot as plt
from math import sqrt

z_ideal = [0, 0]
z_nadir = [sqrt(2), sqrt(2)]

def prob(x):
    return [sqrt((x[0] - 1) ** 2 + x[1] ** 2), sqrt(x[0] ** 2 + (x[1] - 1) ** 2)]
def prob_normalized(x):
    z_ideal = [0, 0]
    z_nadir = [sqrt(2), sqrt(2)]
    z = prob(x) 
    return [(zi - zideali) / (znadiri - zideali) for 
            (zi, zideali, znadiri) in zip(z, z_ideal, z_nadir)]
def plot_sols(coords, title, ind = 1):
    fig = plt.figure(ind)
    plt.scatter([x[0] for x in coords], [x[1] for x in coords])
    plt.title(title)
    plt.show()

In [16]:
import numpy as np
from scipy.optimize import minimize
import ad
import pyomo.core.base.expr as pyoex
def e_constraint_method(f,eps,z_ideal,z_nadir):
    points = []
    for epsi in eps:
        ub = epsi[0] * (z_nadir[1] - z_ideal[1]) + z_ideal[1]
            
        model = ConcreteModel()
        model.x = Var([0, 1])
        model.obj = Objective(expr = pyoex.sqrt((model.x[0] - 1) ** 2 + model.x[1] ** 2))
        model.Constraint1 = Constraint(expr = pyoex.sqrt(model.x[0] ** 2 + (model.x[1] - 1) ** 2) <= ub)
        
        opt = SolverFactory("ipopt")
        res = opt.solve(model)
        points.append([model.x[0].value, model.x[1].value]) # We should check for optimality...
    return points

eps = np.random.random((100,1))
repr = e_constraint_method(prob_normalized, eps, z_ideal, z_nadir)
plot_sols(repr, "Solutions in parameter space", 1)

repr_objspace = [prob(repri) for repri in repr]
plot_sols(repr_objspace, "Solutions in objective space", 2)

Käytin epsilon-constraint -metodia siten, että $f_1$ minimoitiin niin, että $f_2$ oli rajoitettu arvotulla raja-arvolla. Painotusmenetelmä ei tähän tilanteeseen sovi, sillä se löytää käytänössä ainoastaan päätypisteet. Tehtävän yksinkertaisuutta voisi tietysti hyväksikäyttää ja jakaa pisteet täysin tasavälein janalle, koska sen tiedetään sisältävän kaikki Pareto-optimaaliset ratkaisut.